# HPDM097: Working with geospatial health data

Many health service problems are affected by geography.  The first step is often to understand demand from a geospatial perspective.  To do this we can use basic summary statistics, but also make use of Python's extensive mapping and geospatial packages.  For small to medium size geospatial problems you may find this is all that is needed to help your customers/collaborators make a decision.

**In this computer practical you will learn how to:**

* Use `GeoPandas` for managing and importing geospatial health data
* Plot basic and sophisticated maps of health demand and travel times to nearest health delivering facilities to help answer research questions.

In this tutorial we will use a health dataset.

* **Exeter stroke admissions.**  The the number of stroke unit admissions to the Royal Devon and Exeter hospital by Lower Super Output Area

In a geospatial analysis it is sometime useful to bring in additional data to enhance the analysis.  We will use:

* **Population estimates by Lower Super Output Areas**
* **Indicies of multiple deprevation (IMD) by Lower Super Output Area**.  In particular, we will look at deciles of depreviation which enable us to identify the least and most deprived areas. 
* **Deterministic Travel time from LSOAs to hospitals**.  These are data taken from GIS software (in this case Routino).

> At the end of the notebook there are several **optional** exercises that explore creating more sophisticated inset maps and use of `folium`

---

# Imports

> Please run this notebook with the provided environment `hds_logistics`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
import contextily as ctx
import folium

---

# Introduction to GeoPandas

`Geopandas` builds on top of `pandas` and extends its functionality to tackle geographic data. 

> In pandas you work with a `DataFrame`.  In `geopandas` you work with a `GeoDataFrame`.  As these are extensions of a `DataFrame` they behave in a very similar way, but have additional functionality for handling geometry.

## Importing Geospatial LSOA data for the South West of England

For geospatial analysis you usually work with some form of shape file.  We will use shape data stored in GeoJson format.  This is publically available data for England published by the [ONS](https://opendata.arcgis.com/datasets/fd7e9e6e82584a54b06aae40b8ca6988_0.geojson). In this exercise we will work with a local subset of the data for the South West of England.  

Note the way to read the file:

```python
# include "zip://" in the file path as it is a compressed file
gpd.read_file("zip://./data/devon_cornwall.zip")
```

> **Note**: the England file is 1GB+, you will need to wait for it, but it does load fine into geopandas.  The South West dataset is 75mb which is still fairly large so it is compressed. Geopandas can handle compressed data.

Shape files and other data files are usually quite large and not very transferable with your analysis files.  It is straightforward to open a data file from a remote repository such as GitHub.  It may take a few seconds to download if it is a big file!  For example, to load the same data directory from its GitHub repo we just need to specify its url:

```python
url = "https://raw.githubusercontent.com/health-data-science-OR/healthcare-logistics/master/mapping/data/devon_cornwall.zip"
gpd.read_file(url, crs='EPSG:4326')
```
### Coordiante Reference Systems,

Geographic data has a **crs** (Coordinate Reference System). EPSG:27700 is the crs to use when geography is in BNG (British National Grid Eastings and Northings).  Geopandas handles this automatically for you, but occationally you will need to set it during your analysis.

* EPSG:27700 OSGB 1936 / British National Grid -- United Kingdom Ordnance Survey. Co-ordinates are in Eastings (X) and Northings (Y).
* EPSG:4326 WGS 84 -- WGS84 - World Geodetic System 1984. Co-ordinates are in Longitude (X) and Latitude (Y).
* EPSG:3857 - projection for displaying lat/long as a flat map.  We will use this when adding a `contextily` base map.

> The South West data is coded in Lat/Long format

# Exercise 1.  Read in a local GeoJson file

**Task:**
    
* read in the geojson shape data from /data/devon_cornwall.zip and assign to a variable with name `south_west`.  The data has Lat/Long coordinates.

**Questions:**
* What is the the type of `south_west`?
* What is shape of `south_west`?
* What are the columns in `south_west`?
* How do you view the first 5 rows of `south_west`?

In [ ]:
# your code here ...

# Exercise 1.a Read in remote data to a GeoDataFrame

**Task**
* Read shape file data from the provided URL into a `GeoDataFrame` called `remote_sw`

**Hints**:
* This is the identical data to what we have in `south_west`, but downloaded from a remote source. View the first 5 rows of `remote_sw` to see that it matches the data in `south_west`.


In [ ]:
# if you run this it will take some time to download..
url = 'https://raw.githubusercontent.com/health-data-science-OR/' \
        + 'healthcare-logistics/master/mapping/data/devon_cornwall.zip'

#your code here...

# Selecting sub regions

Your `GeoDataFrame` contains a number of fields related to LSOA.  The key ones are

* LSOA11CD: e.g. E01029722.  This is a unqiue code identifying the LSOA defined in 2011. The first charactor 'E' represents the country in this case **E**ngland.
* LSAO11NM: e.g. Stafford 009C.  This is the unique name identifying the LSOA defined in 2011.

> We also have a field LSOA11NMW - this is the name in Welsh.

The name field is useful for quick filtering of the main dataset down to sub regions.  For example to filter for 'Exeter' we use:

```python
region = "Exeter"
exeter = south_west[south_west['LSOA11NM'].str.contains(region)]
```

# Exercise 2: Filtering sub regions

**Task:**

* Create four new `GeoDataFrame` objects for "Exeter", "North Devon", "Cornwall" and (the Isles of) "Scilly".  
* Call the sub regions `exeter`, `north_devon`, `cornwall` and `scilly`, respectively.
* Concatenate `cornwall` and `scilly` into a single `GeoDataFrame`, **using the variable name `cornwall`**.
* Check the shape of your new objects.
* Take a look at the data in each `GeoDataFrame` and confirm your code has worked.

**Hints:**

* Remember a `GeoDataFrame` is an extension of `pandas.DataFrame`.  You can use the same method to **concatenate** each type of object.

In [ ]:
# your code here...

# Plotting maps

The basic plotting of a map is done *pandas style* by calling `GeoDataFrame.plot(**kwargs)`.  For example to plot `south_west` use the following code:

```python
# pandas style plotting 
ax = south_west.plot(figsize=(12, 9))
```

> The method returns a matplotlib `axis` object.  This is used to add additional layers and information etc. 

# Exercise 3: Basic plotting

**Task:**
* Plot `south_west`, `exeter`, `north_devon` and `cornwall` frames.


In [ ]:
# your code here ...

# Loading data representing points of importance and plotting it.

Often with health data we need to add health delivering facilities to a map.  For example acute hospitals, community hospitals, GP surgeries, minor injury units, walk in clinics, or outpatient clinics. In the UK, these will usually have a postcode / exact Lat/Long that isn't confidential or sensitive. It is therefore relatively straightforward to add them to a map.  The tricky part is that we need to create the geometry manually.

We will work with `./data/sw_acute_hospitals.csv`.  This has four columns: acute (name), postcode and lat, long. 

> Since version 1.0.0 of `geopandas.read_file()` will return a `pandas.DataFrame` if no geometry is included.  We therefore need a step that creates the geometry and converts to a `GeoDataFrame`.

**Steps to load and preprocess the data** 
1. Load the `DataFrame` from file.  Let's assume the frame has been given the name `sw_acute`
2. Manually create the geometry field.  Use the `geopandas.points_from_xy()` method like so:

```python
# Set geometry (note longitude comes first as GeoPandas expects x/y geometry)
sw_acute = gpd.GeoDataFrame(
    sw_acute, geometry=gpd.points_from_xy(sw_acute.long, sw_acute.lat), 
    crs="EPSG:4326"
)
```

Once you have loaded and preprocessed you are ready to plot!  

**Steps to plot**:

1. Plot the LSOA shapes contained in `south_west` 
2. Plot the hospitals.  To do this you need to pass in the `axis` object returned in step 1.  Like so:

```python
sw_acute.plot(ax=ax, edgecolor='k', facecolor='w', markersize=200, 
              marker='*')
```

> We have passed in some options here to determine the colouring, size and shape of the points.

# Exercise 4: Plotting acute hospitals in the South West.

**Task**:
* Using `south_west` and the file `./data/sw_acute_hospitals.csv` plot all of the acute hospitals in the south west on a map.
* Feel free to experiment with parameters for colour, markersize and style.
* (Optional) add a label to the plot using standard Matplotlib.

**Hints:**
* Your basic result should look like

![test](./images/basic_sw_map_points.png)

* To add a label to the plot you can use the following code snippet that offsets the 'acute' field name from the point.

```python
# Add labels
for x, y, label in zip(
    sw_acute.geometry.x, sw_acute.geometry.y, sw_acute.acute):
        ax.annotate(label, xy=(x, y), xytext=(8, 8), textcoords="offset points",
                    backgroundcolor="w", fontsize=8)
```
* If you want to drop the Lat/Long axes then use

```python
ax.set_axis_off()
```

In [ ]:
# your code here ...

# Visualising quantitative geospatial data.

We will now work with the stroke admissions dataset.  These are patients admitted to Exeter hospital.  Note that these patients may live outside of the 'exeter geographic area'.  

It is highly likely that your quantitative data will be stored in a seperate file from your spatial shapes. For example, you would expect that the quantitative data is supplied by an NHS Clinical Commissioning Group (CCG) or a provider organisation such as an NHS Acute Trust.  The spatial shape data would need to be sourced independently for example from the UK's Office of National Statistics (ONS; or equivalent from another country). 

A key task in geospatial analysis is therefore merging these datasets.  In order to merge two datasets, each dataset needs to have a common naming system for the areas to enable the algorithm to match the corresponding areas across the datasets. For our example this is contained in the field LSOA11NM. The field LSOA11CD is also often used.

The basic process for doing this is as follows:

1. Load and verify your shape file as `GeoDataFrame`.  Check that this plots as expected.
2. Load and verify your quantitative health data.  An option is to do this as a straightforward pandas `DataFrame`
3. Use the `GeoDataFrame.merge(data, on, how)` method to combine the fields.  It is easiest to have a key in each dataframe with the same name.

**Example merging:**

Assume you have a `DataFrame` with name `exeter_strokes` with columns ['LSOA11NM', 'admissions'] and a `GeoDataFrame` named `south_west` containing the shapes.

```python
gdf_exeter_admit = \
    south_west[['LSOA11CD', 'LSOA11NM', 'geometry']].merge(exeter_strokes, 
                                                           on='LSOA11NM', 
                                                           how='inner')

```
The code listing above linked on the key 'LSOA11NM'. The result is new `GeoDataFrame` with the same number of rows as `exeter_strokes`, the fields 'LSOA11CD', 'LSOA11NM', 'geometry' and 'admissions'.  You get the same number of rows as `exeter_stroke` because the operation was an **inner** join.  As an alternative you could set the parameter `how='outer'`.  An **outer** join would result in the same number of rows as `south_west`.  In this case the 'admissions' field would be `NaN` (not a number aka none or null) for those LSOA's where there were no stroke admissions in Exeter.  

Specify which columns you'd like from each of the DataFrames by including the column names in double square brackets. If you'd like all of the columns to be included in the resulting `GeoDataFrame`, then just write the variable name (without double brackets following). For the example above the resulting merge will take three columns from `south_west` and all the columns from `exeter_strokes`. Always remember to include the common column that the merge is joining on in the selection.

**Example plotting**

Now that you have the merged dataset you can plot it. The following code listing outputs a plot where LSOAs are coloured depending on the number of admissions.

```python
ax = gdf_exeter_admit.plot(column='admissions', figsize=(12,6),  
                           legend=True)
```

The code listing above uses a continuous scale.  Sometimes it is useful to **bin** the values into ranges.  Using the `scheme='` parameter switches to automatically sized bins.  For example using the Quantiles scheme:

```python
ax = gdf_exeter_admit.plot(column='admissions',scheme='Quantiles', 
                           figsize=(12,6),  
                           legend=True)
```

You can try a few options (see help for an exhaustive list)
* Quantiles - bins split by the quantiles
* EqualIntervals - bins into equal sizes
* StdMean - bins split by the number of standard deviations a variable is from the mean.

You might also find that the default colour map (the shading of the areas) is not to your liking or is not clear or helpful for a decision maker.  To can change this using the `cmap` parameter.  A list of options is [here](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html).  But easy ones to remember are 'autumn', 'winter', 'spring', 'summer'.

```python
ax = gdf_exeter_admit.plot(column='admissions',scheme='Quantiles', 
                           figsize=(12,6), cmap='autumn',
                           legend=True)
```

# Exercise 5: Bringing in the admissions data

**Task:**
* Load the stroke admissions dataset and merge it with `south_west` to create a new `GeoDataFrame`.  It it recommended you call the dataframe `gdf_exeter_admit`.
* The data are stored in `data/exeter_stroke_admissions.csv`
* Plot the admissions data
* Add in a marker for the Royal Devon and Exeter hospital

**Questions** 
* Test out the default, Quantiles, EqualIntervals and StdMean schemes.  Which do you prefer and why?
* Test out different colour maps.  Which do you prefer?

In [ ]:
#your code here ...

# Normalising geospatial health data 

A weakness of the analysis we have conducted so far is that the population in each LSOA varies between 1000 and 3000.  At the extremes this means that our 'high and low incidence' LSOAs might just be a function of population size.  At the very least we have a source of variance in the analysis that is potentially counfounding our interpretation.

To get a handle on this you will now use estimates of the population size in each LSOA to normalise the data.  You will calculate the number of admission per 1000 of population.  This is a simple calculation.  If a LSOA has a population of 1500 and there were 50 stroke admissions then the rate per 1000 of population is 50 / (1500 / 1000) = 33.33

In the UK, you can again get estimates of population size from the Office of National Statistics ![(ONS)]() geoportal. The file `mid-2017-lsoa-pop-estimates.zip` contains these data broken down by age band (this is a compressed .csv file).

# Exercise 6: Comparing absolute admissions with rate per 1000

**Task:**
* Read in `mid-2017-lsoa-pop-estimates.zip` (url provided below) to a `DataFrame`and merge the 'All Ages' field  (total count of population) with your `GeoDataFrame`.
* Calculate the rate per 1000 of population. **Call this new column `admits_per_1k`**
* Create a side by side (or above and below) plot of the rate per 1k of population and the absolute number of admissions
* Use your preferred colour map and scheme to display the data.

**Questions:**
* What are the differences between the two approaches to visualisation?
* Are there any LSOA's that have particularly high admission rates per 1000 of pop?

**Hints**
* In matplotlib you can create **subplots** that allow you to plot multiple maps int the same figure.  For example the code:

```python
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(15,12))
```

creates 2 rows and 1 column with a shared x axis.  The variable `ax` is a numpy array of the 2 axes (i.e. it has 2 elements referencing the two subplots).

In [ ]:
# your code here ...
url = 'https://raw.githubusercontent.com/health-data-science-OR/' \
    + 'healthcare-logistics/master/mapping/data/mid-2017-lsoa-pop-estimates.zip'
#...

# Indicies of Multiple Deprivation (IMD)

An Index of Multiple Deprivation (IMD) is an overall measure of multiple deprivation experienced by people living in an area and is calculated for every Lower layer Super Output Area (LSOA) in England. The IMD can be used to rank every LSOA in England according to their relative level of deprivation.  IMD for a LSOA is often viewed as a **decile** (1-10 where 1 is the top 10% of deprived areas in England).  When analysing health service demand and equitable access, IMD can be a useful measure to explore geographically. 

> IMD updates are published periodically.  There were updates in 2010, 2015 and 2019.  

# Exercise 7: Plotting deprivation

**Task:**
* Read `Indices_of_Multiple_Deprivation_(IMD)_2019.zip` (URL provided below) into a `DataFrame` and merge the field `IMD_Decile` with your `GeoDataFrame` 
* Create a side by side (or above and below) plot of the rate per 1k of population and the deprivation
* Try different colour maps and schemes to display the data.

In [ ]:
# your code here ...
url = 'https://raw.githubusercontent.com/health-data-science-OR/' \
        + 'healthcare-logistics/master/mapping/data/' \
            + 'Indices_of_Multiple_Deprivation_(IMD)_2019.zip'

# Adding a base using contextily

You can add a base map using contextily.  We have already imported the package:

```python
import contextily as ctx
```

> To work with contextily at the moment it is best to convert your data into a flat projection i.e. EPSG:3857

Contextily downloads tile maps from internet sources (note this means that there may be a short delay while it downloads).  For example to add an openstreetmap base tile:
    
```python

# convert your data to a flat projection
gdf_exeter_admit = gdf_exeter_admit.to_crs(epsg=3857)
sw_acute = sw_acute.to_crs(epsg=3857)

# create map of IMD deciles
ax = gdf_exeter_admit.plot(column='IMD_Decile',  cmap='autumn',
                       legend=True, figsize=(15,9))

#add base map using contextily
ctx.add_basemap(ax, 
                #set to use open street map
                source=ctx.providers.OpenStreetMap.Mapnik,
                #zoom level 
                zoom=10)
```

Example ctx.providers are:

* "OpenStreetMap.Mapnik"
* "CartoDB.Positron"
* "CartoDB.Voyager"

you can see all provider by running

```python
ctx.providers
```

# Exercise 8

**Task:**
* Run the code below to add a contextily base map
* Try different `source` parameters in `add_basemap`


In [ ]:
# convert the exeter admissions dataset to EPSG=3857
gdf_exeter_admit_flat = gdf_exeter_admit.to_crs(epsg=3857)

In [ ]:
# convert the sw acute hospital data to EPSG=3857
sw_acute_flat = sw_acute.to_crs(epsg=3857)

In [ ]:
# create map of IMD deciles
fig, ax = plt.subplots(1, figsize=(10, 10))

gdf_exeter_admit_flat.plot(column='IMD_Decile',
                          cmap='inferno_r',
                          # slight transparency to see base map
                          alpha = 0.70, ax=ax, 
                          # include colour map legend
                          legend=True,
                          # Adjust size of colourmap key, and add label
                          legend_kwds={'shrink':0.4, 
                                       'orientation': "vertical",
                                       'label':'IMD_Decile'})


#plot the RDE and only
sw_acute_flat.loc[sw_acute['acute'] == 'RDE'].plot(ax=ax, edgecolor='k', 
                                                   facecolor='r', 
                                                   markersize=200, 
                                                   marker='*')

#### CODE UPDATED 2024. all stamen tiles deprecated.
# base map using contextily
ctx.add_basemap(ax,
                source=ctx.providers.OpenStreetMap.Mapnik,
                zoom=10)
#####

ax.set_axis_off()
ax.set_title('IMD Decile by LSOA')
# Adjust for printing
ax.margins(0)
plt.subplots_adjust(left=0.01, right=1.0, bottom=0.0, top=1.0)
ax.apply_aspect()

# Working with travel times and distances

A number of decisions in health are influenced by balancing quality, cost and equitable access to health services.  Planners often use travel time (or distance) to understand the implications of different configurations of their system on fairness.  For example, the combination of outpatient clinics open on a given days, centralisation of specialist services or the closure of a facility.  A map can be a useful to support decision makers understanding of if certain areas 'travel burden' is greater than others.  This might be enhanced for example, by considering deprivation and attempting to ensure that the most deprived areas are not unduly disadvantaged by service changes.

As a simple example of exploring travel times to health facilities recall our example with all of the acute hospitals in the south west. 


In [ ]:
# drop isles of scily as it skews travel times...
south_west2 = south_west[~south_west['LSOA11NM'].str.contains('Scilly')]

# plot
ax = south_west2.plot(figsize=(8, 8))

# add markers for hospitals
ax = sw_acute.plot(ax=ax, edgecolor='k', facecolor='y', markersize=500, 
              marker='*')

# optionally add in labels
for x, y, label in zip(
    sw_acute.geometry.x, sw_acute.geometry.y, sw_acute.acute):
        ax.annotate(label, xy=(x, y), xytext=(8, 8), textcoords="offset points",
                    backgroundcolor="w", fontsize=8)

ax.set_axis_off()

# Exercise 9: Preprocessing and mapping travel times.

You are provided with deterministic estimates of travel times in minutes between all the centroid of all 32,843 LSOAs and hospitals in England in the file 'travel_matrix_minutes.zip' (url provided below).  You can think of this table as a 'lookup'.  The rows list LSOA and the columns list postcodes (hospital) locations. 

**Task:**
* Load the national travel time matrix into a `DataFrame`
* For the LSOAs and hospitals in the south west find the closest hospital (shortest travel time to a south west hospital).
* Revise the above plot to create a geospatial view of travel time to hospital.

**Hints**:
* The travel time data is a compressed .csv file.  Pandas can handle zip files automatically.  Just called `pd.read_csv(url)`
* To help, you can work with a 'trimmed' down or 'local' version of the travel matrix relating only to the south west hospitals.  To do this you can use some standard `DataFrame` operations.  Call this variabe `local_travel_matrix`. Remember that you have `DataFrame` called `sw_acute` that contains a column of their postcodes.  
* Once you have limited to the hospitals in the south west you can find the minimum travel time using standard pandas (just remember that you are working with columns so axis=1).
```

In [ ]:
# your code here...
url = 'https://raw.githubusercontent.com/health-data-science-OR' \
    + '/healthcare-logistics/master/mapping/data/travel_matrix_minutes.zip'


# Evaluating travel time to multiple health delivering facilities

It is common to use the following approaches to assess travel time to health delivery facilities

* Average travel time
* Weighted average travel time (i.e. weighted by the number of cases in each LSOA).
* Maximum or a percentile (e.g. 95th) of travel time 
* Proportion of demand within x minutes 

# Exercise 10

**Task**:
* Use the function `generate_synthetic_demand` (given below) to generate some demand data.
* By extending the code below calculate the average, demand weighted average, maximum and 95th percentile of car travel times to each hospital
* Repeat the analysis but limit it to three hospitals with postcodes 'EX2 5DW', 'EX31 4JB', 'TR1 3LQ'
* Optional: display the results in a well presented chart or table.

**Hints**:
* A fast way to find a hosptial travel time average, maximum value or percentile is to used the `DataFrame.groupby()` method.  

```python
lsoa_demand.groupby(by='nearest_facility')['mins_to_nearest_hospital'].mean()
```

* To calculate a weighted average travel time is a bit more involved in standard pandas.  It works best if you use numpy's `average` function that accepts weights.  To implement this with a lambda expression you use the following code:

```python
lsoa_demand.groupby(by='nearest_facility').apply(lambda x: 
                                                 np.average(
                                                     x.mins_to_nearest_hospital, 
                                                     weights=x.cases))
```



In [ ]:
def generate_synthetic_demand(lsoa_frame, random_seed=42):
    # create random num generator
    rng = np.random.RandomState(random_seed)
    
    # code to generate synthetic data
    lsoa_demand = lsoa_frame['LSOA11NM'].to_frame()
    lsoa_demand['cases'] = rng.randint(0, 20, size=len(lsoa_demand))
    
    return lsoa_demand

In [ ]:
lsoa_demand = generate_synthetic_demand(local_travel_matrix)

In [ ]:
lsoa_demand.info()

In [ ]:
lsoa_demand.head()

In [ ]:
# your code here ...

----
# **Optional additional content**
## To work through in your own time

The following additional content has been provided for further study.

* Adding an inset map to zoom in on a specific area
* Using folium to create an interactive map.

---


# Adding an inset map to zoom in on a specific area

A limitation of your stroke admissions map is that the areas with higher density have smaller LSOA shapes.  In particular it is very difficult to see the detail in Exeter.  One way to deal with varying size in a static map image is to create a second 'close up' map of Exeter as a matplotlib inset of your map.  You can think of this as a figure within a figure.  The example we will code is below:
![INSERT IMAGE](images/insert_image.png)

## Step 1: Filter for admissions from Exeter LSOAs
To build our insert map for admissions per 1k of population in Exeter we first need to limit our `GeoDataFrame` to data only containing Exeter LOSAs. There are a couple of way to achieve it, but a simple approach is to filter by 'LSOA11NM' as you learnt eariler.

In [ ]:
# create a new GeoDataFrame containing only exeter LSAOs
region = "Exeter"
mask = gdf_exeter_admit['LSOA11NM'].str.contains(region)
exeter_city_rate = gdf_exeter_admit[mask]
exeter_city_rate.plot(figsize=(12,6), column='admits_per_1k')

## Step 2: Remembering that the shading needs to be consistent across the main map and its inset

Remember that the inset map is a close-up of an area in your main map.  It therefore needs to have consistent colouring for admission numbers.  To achieve this you should make use of the `'userdefined'` scheme.  This allows you to pass a list of user defined bins as well as their labels. The following code demonstrates how this works.

In [ ]:
def bins_to_labels(bins, insert_one_as_min=True):
    '''
    Helper function that creates a list of labels
    for bins presented in a GeoDataFrame plot.
    
    Params:
    -------
    bins - list
        List of floats to group data
        
    insert_one_as_min - bool, optional (default=True)
        Used if you are only including LSOAs that have
        at least 1 admission.  Inserts a minimum of 1.
    '''
    if insert_one_as_min:
        bins = bins.copy()
        bins.insert(0, 1)
    return [str(bins[i]) + '-' + str(bins[i+1]) for i in range(0, len(bins)-1)]

In [ ]:
# example usage
bins = [2, 10, 15, 20]
print(bins_to_labels(bins))

print(bins_to_labels(bins, insert_one_as_min=False))

In [ ]:
# userdefined scheme example limited to Exeter
bins = [5, 10, 15, 20]
bin_labels = bins_to_labels(bins)


# note the scheme = 'userdefined' and 
# bins are passed via classification_kwds
ax = exeter_city_rate.plot(column='admits_per_1k',
                           scheme='userdefined', 
                           cmap='Spectral',
                           figsize=(12,9),
                           legend=True, 
                           classification_kwds={'bins': bins},
                           legend_kwds={'bbox_to_anchor':(1.2, 1.05),
                                        'fontsize':16,'frameon':False,
                                        'labels':bin_labels})
ax.set_title("Exeter City")
ax.set_axis_off()

## Step 3: Creating an inset map

Matplotlib makes this relatively straight forward.  The inset map API has changed recently and I recommend using the new approach below as the old method will be deprecated in future releases.  

The basic logic requires you to create a map and then use `Axes.inset_axes` to create a the inset axis within the main axis.  Full documentation and links to other examples can be found in the [main documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.inset_axes.html#matplotlib.axes.Axes.inset_axes)

It is alos possible to add "zoom lines" from the main map to the inset.  To do this you need to use the `Axis.indicate_inset_zoom` method.
 
# Optional Exercise 1
The code below generates an inset map of Exeter and includes explanation comments.

**Task**:
* Run the code below and explore the different parameters to see how it affects the map.

**Hints**
* Maybe copy-paste the code so you still have the original!

In [ ]:
# setup custom bins - note it is a good idea to know what the min/max 
# of demand per LSOA are.
bins = [5, 10, 15, 20, 25, 30]
bin_labels = bins_to_labels(bins)

cmap = 'viridis'

# create figure and main axes objects
fig, ax1 = plt.subplots(figsize=(12,12))

# plot data (excluding the outlier)
ax1 = gdf_exeter_admit.plot(column='admits_per_1k',scheme='userdefined', 
                            cmap=cmap,
                            legend=True, 
                            classification_kwds={'bins': bins},
                            legend_kwds={'bbox_to_anchor':(0.85, 0.97),
                                        'fontsize':14,'frameon':False,
                                        'labels':bin_labels,
                                        'title':'RD&E stroke admissions'
                                                +'\nper 1000 of population',
                                        'title_fontsize':14},
                             ax=ax1)


#### UPDATED code for 2024. New simpler inset API
# create the inset
# set the lower left x, y location and relative size of the bounding box
# note these are in fractional units.
ax2 = ax1.inset_axes([0.025,0.5,0.45,0.45])

#####

#plot the exeter LSOA inset
ax2 = exeter_city_rate.plot(column='admits_per_1k',
                           scheme='userdefined', 
                           cmap=cmap,
                           legend=False, 
                           classification_kwds={'bins': bins},
                           ax=ax2)


# manual text within inset
ax2.text(-3.575, 50.755,'Exeter detail', fontsize='large')


# plot the RDE and only
ax2 = sw_acute.loc[sw_acute['acute'] == 'RDE'].plot(ax=ax2, edgecolor='k', 
                                                   facecolor='r', 
                                                   markersize=200, 
                                                   marker='*')

# formatting tweaks...

# align legend left
leg = ax1.get_legend()
leg._legend_box.align = "left"

# set axis 1 limits to provide enough space for exeter
# this might take a bit of trial and error
ax1.set_ylim(50.5, 51.6)
ax1.set_xlim(-4.5, -2.75)

# hide axis tick marks
ax2.tick_params(axis=u'both', which=u'both',length=0)

# hide axis lat long labels, but keep bounding box
ax2.set_xticklabels([])
ax2.set_yticklabels([])

# #### UPDATED CODE 2024 - new interface
# this draw a box around the area where extra detail is needed and
# 'zoom out' lines to the detail.
# this is optional and the chart might look better without it.
# ls = line style '-' for solid, '--' for dashed
# ec = edge colour
# fc = face colour 
ax1.indicate_inset_zoom(ax2, edgecolor="black", ls='--', ec='black', fc=None)
# ####

# hide the box and axis labels
ax1.set_axis_off()

fig.savefig('images/insert_image.png', bbox_inches='tight')

---
# Using Folium to create interactive plots

One of the limitations of the geospatial analysis we have done so far is that we must manually drill down to find out more information (other fields) about a LSOA.  When viewing all LSOAs, it is difficult to see the LSOAs in Exeter itself.  This is due to population density - the higher density areas are smaller to maintain the 1000-3000 population of a LSOA.

Folium and Choropleth allow us to create an interactive map where we can both include additional information in a 'tooltip' and control the level of zoom on a map.  

## Creating an displaying a Folium map

Create a folium base map as follows:

```python
#create the map object
stroke_map = folium.Map(location=[50.71671, -3.50668], zoom_start=9, 
                        tiles='cartodbpositron')

#display the map in the Jupyter notebook
stroke_map
```

The location here is in Lat and Long coordinates (these are the Royal Devon and Exeter Hospital).  They are used to centre the map. The `zoom_start` parameter might take a bit of tweaking to optimise for your application - the higher the number the higher the zoom.  `tiles` is a `str` field where you can specifiy the style of the map.  The following are built into folium

* “OpenStreetMap”
* “Mapbox Bright” (Limited levels of zoom for free tiles)
* “Mapbox Control Room” (Limited levels of zoom for free tiles)
* “Cloudmade” (Must pass API key)
* “Mapbox” (Must pass API key)
* “CartoDB” (positron and dark_matter)

A useful trick is to add a number of Tile layers to the map that can be changed interactively.  

```python
#add some different background layers
tiles = ['cartodbpositron', 'cartodbdark_matter', 'openstreetmap']
for tile in tiles:
    folium.TileLayer(tile).add_to(stroke_map)

#add layer control to map
folium.LayerControl().add_to(stroke_map)
```

# Optional Exercise 2: A basic folium map

**Task:**

* Using folium create a base map
* centre the map on the Royal Devon and Exeter Hospital: Lat, Long: 50.71671, -3.50668
* Add in number of tile layers of your choosing.

**Hints**:
* In the resulting map cell you will see the layer control in the top right hand corner.  Use it to select the different background tiles you have included.

In [ ]:
# your code here ...

## Visualising LSOA shape in folium

To visualise shape in folium we will use **Choropleth**.  This provides an easy to use interface for adding shapes to a map.

The code listing below demonstrates basic usage.

# Optional Exercise 3

**Task**
* run the code below to add a Choropleth map to your base map.  
* explore the effect of changing the parameters e.g. displaying different data fields

**Hints**:
* Other `fill_color` values include ‘YlGnBu’, 'BuPu’,'OrRd’, 'RdPu’.  The colormap options from matplotlib are not available here. 

In [ ]:
#create base map
stroke_map = folium.Map(location=[50.71671, -3.50668], zoom_start=9, 
                        tiles='cartodbpositron')

#### Updated code 2024
# add some different background layers
tiles = ['cartodbpositron', 'cartodbdark_matter', 'openstreetmap']
####

for tile in tiles:
    folium.TileLayer(tile).add_to(stroke_map)

# filter data to display
fields = ['LSOA11NM', 'admissions', 'admits_per_1k', 'IMD_Decile']
lsoa_tia = gdf_exeter_admit[fields]

# field to color; 0 = LSOA11NM; 1 = 'admissions' etc.
field_index = 3

# create and add choropleth map
choropleth = folium.Choropleth(
    # pass the GeoDataFrame
    geo_data=gdf_exeter_admit,
    # Data for overlaying
    data=lsoa_tia,
    #c olumns = [key, field to plot]
    columns=['LSOA11NM', fields[field_index]],
    key_on='feature.properties.LSOA11NM',
    # choose the fill colour
    fill_color='OrRd',
    # set the legent name
    legend_name=fields[field_index],
    # highlight the LSOA shape when mouse pointer enters it
    highlight=True,
    smooth_factor=0).add_to(stroke_map)    
    
# add layer control to map - make sure you do this after adding choropleth map
folium.LayerControl().add_to(stroke_map)

stroke_map

# Adding tooltips in folium

A neat interactive feature of folium is the ability to add a tooltip.  This can display additional information about a lower super output area when the mouse pointer enters its boundary.

For example, to add all of our information so far as a tooltip you need to add the following to our previous folium code.

```python
# Display Region Label
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['LSOA11NM', 'admissions', 'admits_per_1k', 'IMD_Decile'], labels=True)
)

```

# Optional Exercise 4:

**Task**:
* Add a tooltip displaying 'LSOA11NM', 'admissions', 'admits_per_1k',  'IMD_Decile' to your folium map.
* run your code and verify that it has worked.

In [ ]:
# your code here ...